<a href="https://colab.research.google.com/github/JuJuVitoria/AI-gerador-de-ideias/blob/main/projeto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q google-genai

In [ ]:
import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [ ]:
from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [ ]:
from IPython.display import HTML, Markdown

In [ ]:
from google.genai import types

In [ ]:
chat_config = types.GenerateContentConfig(
    system_instruction = """
    Você é um assistente de bate-papo prestativo e informativo, especializado em responder perguntas sobre os recursos de estudo e os próximos passos sugeridos pelo Agente 'GuiaDeRecursos'.

    Seu objetivo principal é:
    - Ajudar o usuário a entender o conteúdo dos sites, vídeos, documentos e cursos que foram recomendados.
    - Esclarecer dúvidas sobre as sugestões de ações iniciais para começar a explorar a ideia de projeto escolhida.
    - Fornecer informações adicionais básicas relacionadas aos temas e tecnologias mencionadas nos recursos.
    - Manter um tom amigável e encorajador para apoiar o aprendizado do usuário.

    Instruções específicas:
    1.  Contexto da Conversa: Você terá acesso à lista de recursos e próximos passos que foram previamente apresentados ao usuário pelo Agente 'GuiaDeRecursos'. Utilize essas informações como contexto para suas respostas.
    2.  Receber Perguntas do Usuário: Aguarde as perguntas do usuário sobre os recursos ou os próximos passos.
    3.  Responder de Forma Clara e Concisa: Forneça respostas diretas e fáceis de entender. Evite jargões técnicos excessivos, a menos que o usuário demonstre conhecimento prévio.
    4.  Basear-se nos Recursos: Priorize responder às perguntas com base nas informações contidas nos recursos que foram fornecidos. Se a resposta não estiver clara nos recursos, você pode fornecer informações gerais básicas sobre o tema.
    5.  Incentivar a Exploração: Encoraje o usuário a explorar os próprios recursos para encontrar respostas e a dar os primeiros passos em seu projeto.
    6.  Oferecer Ajuda Adicional (com moderação): Se o usuário tiver dúvidas mais complexas que não estão diretamente relacionadas aos recursos iniciais, você pode oferecer sugestões gerais de onde procurar mais informações (ex: "Para uma compreensão mais aprofundada, sugiro pesquisar por...", "Você pode encontrar mais detalhes na documentação de...").
    7.  Lembrar suas Limitações: Você não é um especialista em todas as áreas. Se uma pergunta for muito complexa ou fora do escopo dos recursos iniciais, informe o usuário gentilmente que sua especialidade é auxiliar com os materiais fornecidos para o pontapé inicial.
    8.  Manter a Conversa Engajada: Faça perguntas abertas para entender melhor a dúvida do usuário e garantir que sua resposta seja útil.

    Exemplo de interação:
    Agente 'GuiaDeRecursos': "Comece assistindo ao primeiro módulo do curso online sobre Redes Neurais."
    Usuário: "O que são redes neurais?"
    Você (Chatbot): "Redes neurais são modelos computacionais inspirados no funcionamento do cérebro humano. O curso deve abordar os conceitos básicos no primeiro módulo. Você já teve a oportunidade de começar a assistir?"
    Usuário: "Sim, mas ainda não entendi o que é um neurônio artificial."
    Você (Chatbot): "Um neurônio artificial é a unidade básica de uma rede neural. Ele recebe entradas, realiza um cálculo simples e produz uma saída. O curso provavelmente explicará isso com mais detalhes, geralmente usando analogias. Qual parte específica sobre o neurônio artificial está te confundindo?"

    Lembre-se de integrar este agente de chatbot ao seu fluxo após o Agente 'GuiaDeRecursos' fornecer os materiais iniciais. O objetivo é oferecer suporte imediato para as primeiras dúvidas do usuário. Dada a hora, concentre-se em ter um prompt claro e conciso para este agente, mesmo que a implementação completa do chatbot seja básica.
    Boa sorte com a submissão!
    """,
)

In [ ]:
chat = client.chats.create(model = MODEL_ID, config = chat_config)

In [ ]:
!pip install -q google-adk

In [ ]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [ ]:
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [ ]:
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

## Agente de pesquisa de tendências atuais

In [ ]:
def agente_pesquisa_tendencias_atuais(area_interesse, data):
    pesquisador = Agent(
      # Nome do meu agente
      name="agente_pesquisa_tendencias_atuais",
      # Modelo utlizado
      model="gemini-2.0-flash",
      # Instruções
      instruction="""
      Objetivo: Identificar os temas, tecnologias, metodologias e necessidades mais relevantes e atuais na área de interesse especificada pelo usuário.
      Você receberá como entrada a área de atuação ou o campo em que o usuário deseja gerar ideias (ex: "marketing digital", "desenvolvimento de aplicativos", ...).
      Utilize obrigatoriamente a ferramenta google_search para buscar informações atualizadas sobre a área de até no máximo 4 meses antes data passada.
      As buscas devem incluir:
      - Artigos de notícias e publicações recentes.
      - Blogs e influenciadores relevantes da área.
      - Tendências de mercado e relatórios da indústria.
      - Discussões em fóruns e comunidades online.
      - Casos de sucesso e projetos recentes de destaque.
      Foque em identificar o que as pessoas e empresas estão falando e fazendo *agora* nesta área. Quais são os principais problemas que estão tentando resolver ou as novas oportunidades que estão surgindo?
      Extrair Temas Chave: Analise os resultados da pesquisa para identificar os principais temas, desafios, oportunidades e tecnologias que estão em evidência na área.
      Sintetizar Insights: Organize as informações coletadas em um resumo conciso, listando os 5-7 temas atuais mais relevantes, com uma breve descrição de cada um e por que são considerados importantes no momento.
      Formatar Saída: Apresente os temas de forma clara e organizada para o próximo agente.
      """,
      # Descrição do modelo
      description="Agente que pesquisa sobre o tema - tendenciais atuais",
      # Ferramentas
      tools=[google_search]
    )
    entrada_pesquisador_momento = f"Área de interesse:{area_interesse},Data da pesquisa:{data}"
    pesquisa_tendencias_atuais = call_agent(pesquisador, entrada_pesquisador_momento)
    return pesquisa_tendencias_atuais

# Agente pesquisador de tendências futuras

In [ ]:
def agente_pesquisa_tendencias_futuras(area_interesse, data):
    pesquisador_futuro = Agent( # Use a different variable name to avoid confusion
      # Nome do meu agente
      name="agente_pesquisa_tendencias_futuras", # Update the agent name
      # Modelo utlizado
      model="gemini-2.0-flash",
      # Use 'instruction' instead of 'instructions'
      instruction="""
      Objetivo: Prever e identificar as tendências futuras, tecnologias emergentes e desafios potenciais na área de interesse especificada pelo usuário.
      Você receberá como entrada a área de atuação ou o campo em que o usuário deseja gerar ideias (ex: "inteligência artificial", "biotecnologia", ...).
      Utilize obrigatoriamente a ferramenta google_search para buscar informações sobre:
      - Relatórios de pesquisa e desenvolvimento.
      - Análises de mercado de longo prazo.
      - Patentes recentes e inovações.
      - Publicações científicas.
      - Opiniões de especialistas e futuristas.
      - Tendências demográficas e sociais que podem influenciar o futuro da área.
      Além de relatórios e patentes, pense em como mudanças sociais, econômicas ou avanços em outras tecnologias (como IA, biotecnologia, etc.) podem impactar esta área no futuro. O que as pesquisas acadêmicas e os especialistas apontam como as próximas grandes coisas?
      Extrair Insights Futuros: Analise os resultados da pesquisa para identificar os principais temas, tecnologias, metodologias e desafios que provavelmente serão relevantes na área nos próximos anos.
      Sintetizar Previsões: Organize as informações coletadas em um resumo conciso, listando os 5-7 temas futuros mais promissores, com uma breve descrição de cada um e por que são esperados para serem importantes.
      Formatar Saída: Apresente os temas de forma clara e organizada para o próximo agente.
      """,
      # Descrição do modelo
      description="Agente que pesquisa sobre o tema - tendencias futuras", # Update the description
      tools=[google_search]
    )
    entrada_pesquisador_futuro = f"Área de interesse:{area_interesse},Data da pesquisa:{data}"
    pesquisa_tendencias_futuras = call_agent(pesquisador_futuro, entrada_pesquisador_futuro)
    return pesquisa_tendencias_futuras

# Agentes gerador de ideias

In [ ]:
def gerador_ideias(tentencias_atuais, tendencias_futuras):
  gerador = Agent(
      name="gerador_de_ideias",
      model="gemini-2.0-flash",
      instruction="""
      Você receberá como entrada a lista de temas atuais (do Agente 1) e as tendências futuras (do Agente 2).
      Com essas ideias você irá criar um Brainstorming: Para cada tema atual e cada tendência futura, utilize o Google Gemini para gerar o máximo de ideias de projetos possível. Incentive a criatividade e a exploração de soluções não convencionais.
      Seja ousado! Tente gerar ideias que misturem temas de formas inesperadas. E se uma tecnologia futura fosse aplicada para resolver um problema atual de uma forma completamente nova? Pense em 'e se...?' para estimular a criatividade.
      Combinar Temas e Tendências: Explore a possibilidade de gerar ideias que estejam na interseção de temas atuais e tendências futuras, buscando projetos que possam ser relevantes tanto agora quanto no futuro.
      Variedade de Ideias: Busque gerar ideias de diferentes tipos e escalas (pequenos projetos de portfólio, iniciativas maiores, ferramentas, plataformas, etc.).
      Formatar Saída: Apresente uma lista não filtrada de todas as ideias geradas, indicando o tema atual ou a tendência futura que as inspirou.
      """,
      description="Gerador de ideais para tendências atuais e futuras"
  )
  entrada_gerador = f"Tendências atuais: {tentencias_atuais}\nTendências futuras: {tendencias_futuras}"
  gerando_ideias = call_agent(gerador, entrada_gerador)
  return gerando_ideias

# Agente avaliador de ideias

In [ ]:
def avaliador_de_projetos(ideias_projetos):
    avaliador = Agent(
        name="avaliador_das_ideias_de_projetos",
        model="gemini-2.0-flash",
        instruction="""
        Você terá o papel de ser um crítico. Seu objetivo será avaliar as ideias de projetos geradas pelo Agente 3 com base em critérios predefinidos e fornecer um ranking das ideias mais promissoras para o usuário.
        Você agente receberá a lista de ideias geradas pelo Agente 3, onde você irá avaliar cada ideia com base nos seguintes critérios:
        - Relevância: Quão relevante a ideia é para os temas atuais e/ou futuros identificados?
        - Inovação: Quão original e criativa é a ideia? Apresenta uma nova abordagem ou solução?
        - Viabilidade: Quão factível é a implementação do projeto, considerando os recursos e tecnologias disponíveis? (Lembre-se que o foco é inspiração, mas uma ideia completamente irreal pode ser menos útil).
        - Potencial de Impacto: Qual o potencial do projeto para gerar valor, resolver um problema ou criar algo significativo?
        - Dificuldade Estimada: Qual a estimativa de dificuldade para implementar o projeto (baixa, média, alta)?
        Para melhor avaliação dessas ideias você deverá criar um ranking com pontuações de 1 a 10.
        Com base no ranking com as pontuações, você devera rankear as ideias de projetos, destacando as 5 ideias mais promissoras, juntamente com a justificativa da sua avaliação para cada uma.
        Apresente o ranking das ideias de forma clara, mostrando o título da ideia e um breve resumo da avaliação (pontuações, se aplicável, e justificativa), também apresente uma breve tabela sobre sua avaliação.
        """,
        description="Avaliador dos projetos"
    )
    entrada_avaliador = f"Ideias para os projetos {ideias_projetos}"
    avaliador_ideias = call_agent(avaliador, entrada_avaliador)
    return avaliador_ideias

# Agente guia de recursos

In [ ]:
def guia_de_recursos(tema, pesquisa_do_tema, escolha_usuario):
  mentor = Agent(
      name="orientador",
      model="gemini-2.0-flash",
      instruction="""
      Você receberá as seguintes informações:
      - tema: O tema geral da área de interesse do usuário (ex: "marketing digital", "desenvolvimento de aplicativos").
      - pesquisa_do_tema: Os resultados da pesquisa inicial sobre o tema, realizada pelos Agentes 1 e 2 (esta informação pode ser usada para contextualizar a pesquisa, mas o foco principal será na escolha do usuário).
      - escolha_usuario: O número correspondente à ideia de projeto que o usuário selecionou no ranking fornecido pelo Agente 4.

      Você precisa identificar a Ideia Escolhida: Utilize a escolha_usuario para identificar o título ou uma breve descrição da ideia de projeto correspondente no ranking. O foco da pesquisa será o tema específico desta ideia escolhida.
      Utilize a ferramenta [google_search] para realizar uma pesquisa focada no tema central da ideia de projeto escolhida pelo usuário.
      A pesquisa deve visar encontrar recursos práticos para iniciantes e materiais introdutórios relacionados diretamente à ideia selecionada.

      Tente incluir termos como:
      - "tutorial básico de [tema da ideia escolhida]"
      - "primeiros passos em [área relacionada à ideia escolhida]"
      - "guia para iniciantes em [tecnologia chave da ideia escolhida]"
      - "introdução a [conceito fundamental da ideia escolhida]"
      - "melhores cursos online para aprender [habilidade necessária para a ideia escolhida]"
      Avalie os resultados da pesquisa e selecione os 3-5 recursos que melhor se encaixam como um ponto de partida para o usuário aprender sobre a ideia específica que ele escolheu.

      Priorize:
      - Conteúdo introdutório que explica os fundamentos da ideia escolhida.
      - Guias práticos e tutoriais que ensinam como começar a implementar algo relacionado à ideia escolhida.
      - Plataformas de aprendizado com cursos ou documentação sobre os temas da ideia escolhida.
      - Vídeos explicativos que facilitam a compreensão inicial dos conceitos da ideia escolhida.
      Ao buscar recursos, priorize aqueles que ofereçam algo prático, como tutoriais passo a passo, exemplos de código, exercícios simples ou projetos 'hello world'. O objetivo é que o usuário possa 'colocar a mão na massa' rapidamente.

      Para cada recurso selecionado, forneça:
      - Título: O título do recurso encontrado.
      - Breve Descrição: Um resumo conciso do que o recurso oferece e por que ele é relevante para começar a aprender sobre a ideia de projeto específica escolhida pelo usuário.
      - Link (se aplicável): O link direto para o website, vídeo ou documento.
      - Tipo de Recurso: Indique se é um site, vídeo, documento, curso, etc.

      Ofereça ao usuário 2-3 sugestões de ações concretas que ele pode tomar imediatamente para começar a explorar a ideia de projeto específica que escolheu.
      Seja específico e prático, por exemplo:
      - "Comece assistindo ao primeiro módulo do curso online sobre [tema específico da ideia escolhida]."
      - "Leia a introdução da documentação oficial de [tecnologia chave da ideia escolhida]."
      - "Procure por um tutorial simples de 'olá mundo' em [linguagem de programação relacionada à ideia escolhida]."
      - "Explore os primeiros 5 minutos do vídeo explicativo sobre [conceito fundamental da ideia escolhida]."

      Apresente os recursos e os próximos passos de forma clara, concisa e organizada, utilizando marcadores ou listas para facilitar a leitura.
      Incentive o usuário a começar por esses materiais focados na ideia que ele demonstrou interesse.
      """,
      description="Agente guia de recursos",
      tools=[google_search]
  )
  entrada_orientador=f"Tema escolhido pelo usuário: {tema}\nPesquisa feita pelo agente anterior: {pesquisa_do_tema}\n Ideia escolhida pelo usuário: {escolha_usuario}"
  orientador_proximos_passos = call_agent(mentor, entrada_orientador)
  return orientador_proximos_passos

## Chat bot

In [ ]:
def interacao_chatbot(chat):
    print("\nBem-vindo ao Chatbot de Apoio ao para seu Projeto")
    print("Digite sua pergunta sobre os recursos e próximos passos (ou 'fim' para sair).")

    prompt_usuario = input("Sua pergunta: ")

    while prompt_usuario.lower() != "fim":
        try:
            resposta = chat.send_message(prompt_usuario)
            display(to_markdown(resposta.text))
        except Exception as e:
            print(f"Ocorreu um erro ao enviar a mensagem: {e}")

        print("\nDigite sua próxima pergunta (ou 'fim' para sair).")
        prompt_usuario = input("Sua pergunta: ")

    print("\nObrigado por usar o Chatbot de Apoio!")

# Funcionamento

In [ ]:
data = date.today().strftime("%d/%m/%Y")

print("🚀 E aí! Vamos botar a mão na massa e criar um projeto irado com a ajuda dos nossos agentes! 🚀")

while True:
  area_interesse = input("Primeiro, qual a área que te interessa para criarmos projetos? ")
  if area_interesse == "":
    print("Opa, preciso saber a área! Se quiser parar por aqui, só digitar 'fechar'.")
  elif area_interesse.lower() == "fechar":
    print("Fechou! Até a próxima.")
    break
  else:
    break

print(f"\nBeleza! Estamos dando uma olhada no que tá rolando *agora* em {area_interesse}...")
tendencias_atuais = agente_pesquisa_tendencias_atuais(area_interesse, data)

print("Show! E para ter ideias ainda melhores, vamos espiar um pouco o que vem por aí... Pesquisando tendências futuras!")
tendencias_futuras = agente_pesquisa_tendencias_futuras(area_interesse, data)

print("\nCom essas informações em mãos, nossos agentes estão tendo um monte de ideias! Segura aí...")
ideias = gerador_ideias(tendencias_atuais, tendencias_futuras)
display(to_markdown(ideias))

print("Peraí, não acabou! Agora vamos ser um pouco críticos e avaliar essas ideias para ver quais se destacam mais.")
avaliacao = avaliador_de_projetos(ideias)
display(to_markdown(avaliacao))

print("""
    E aí, o que achou? Espero que tenha curtido as ideias e a nossa avaliação!

    Curtiu alguma em especial e quer saber como dar os primeiros passos?
    Se sim, escolhe aí o número da ideia no ranking (de 1 a 5) que você mais gostou.
    Se quiser parar por aqui, só digitar 0.
""")
while True:
    try:
      escolha_user = int(input("Minha escolha é a número: "))
      if 1 <= escolha_user <= 5:
        print(f"\nÓtima escolha! Vamos buscar uns recursos para te ajudar a começar na ideia {escolha_user}...")
        recursos = guia_de_recursos(area_interesse, ideias, escolha_user)
        break
      elif escolha_user == 0:
        print("Fechando por aqui. Qualquer coisa, é só chamar de novo!")
        break
      else:
        print("Essa opção não rolou. Escolhe um número entre 1 e 5, ou 0 pra fechar.")
    except ValueError:
      print("Hmm, parece que você não digitou um número. Tenta de novo!")

if escolha_user != 0:
    display(to_markdown(recursos))

    print("\nE aí, curtiu os recursos? Quer trocar uma ideia sobre eles ou sobre o projeto que escolheu?")
    while True:
      interacao = input("Pra gente bater um papo com o chatbot, digita S (Sim) ou N (Não): ")
      if interacao.lower() == "s":
        interacao_chatbot(chat) # Passa o objeto chat aqui
        break
      elif interacao.lower() == "n":
        print("Beleza! Se precisar de algo, é só rodar o código de novo.")
        break
      else:
        print("Não entendi. Digita S pra sim ou N pra não.")

🚀 E aí! Vamos botar a mão na massa e criar um projeto irado com a ajuda dos nossos agentes! 🚀

Beleza! Estamos dando uma olhada no que tá rolando *agora* em Engenharia de Software...
Show! E para ter ideias ainda melhores, vamos espiar um pouco o que vem por aí... Pesquisando tendências futuras!

Com essas informações em mãos, nossos agentes estão tendo um monte de ideias! Segura aí...


Exception in thread Thread-163 (_asyncio_thread_main):
Traceback (most recent call last):
  File "/usr/lib/python3.11/threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.11/threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.11/dist-packages/google/adk/runners.py", line 138, in _asyncio_thread_main
    asyncio.run(_invoke_run_async())
  File "/usr/lib/python3.11/asyncio/runners.py", line 190, in run
    return runner.run(main)
           ^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/asyncio/runners.py", line 118, in run
    return self._loop.run_until_complete(task)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/asyncio/base_events.py", line 654, in run_until_complete
    return future.result()
           ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/google/adk/runners.py", line 126, in _invoke_run_async
    async for event in self.run_async(
  File "/

ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-473' coro=<AsyncClient.aclose() done, defined at /usr/local/lib/python3.11/dist-packages/httpx/_client.py:1978> exception=RuntimeError('Event loop is closed')>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/httpx/_client.py", line 1985, in aclose
    await self._transport.aclose()
  File "/usr/local/lib/python3.11/dist-packages/httpx/_transports/default.py", line 406, in aclose
    await self._pool.aclose()
  File "/usr/local/lib/python3.11/dist-packages/httpcore/_async/connection_pool.py", line 353, in aclose
    await self._close_connections(closing_connections)
  File "/usr/local/lib/python3.11/dist-packages/httpcore/_async/connection_pool.py", line 345, in _close_connections
    await connection.aclose()
  File "/usr/local/lib/python3.11/dist-packages/httpcore/_async/connection.py", line 173, in aclose
    await self._connection.aclose()
  File "/usr/local/lib/pyth

Peraí, não acabou! Agora vamos ser um pouco críticos e avaliar essas ideias para ver quais se destacam mais.


Exception in thread Thread-165 (_asyncio_thread_main):
Traceback (most recent call last):
  File "/usr/lib/python3.11/threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.11/threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.11/dist-packages/google/adk/runners.py", line 138, in _asyncio_thread_main
    asyncio.run(_invoke_run_async())
  File "/usr/lib/python3.11/asyncio/runners.py", line 190, in run
    return runner.run(main)
           ^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/asyncio/runners.py", line 118, in run
    return self._loop.run_until_complete(task)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/asyncio/base_events.py", line 654, in run_until_complete
    return future.result()
           ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/google/adk/runners.py", line 126, in _invoke_run_async
    async for event in self.run_async(
  File "/


    E aí, o que achou? Espero que tenha curtido as ideias e a nossa avaliação!

    Curtiu alguma em especial e quer saber como dar os primeiros passos?
    Se sim, escolhe aí o número da ideia no ranking (de 1 a 5) que você mais gostou.
    Se quiser parar por aqui, só digitar 0.
    
